<a href="https://colab.research.google.com/github/NgPcAnhh/NCKH-2025/blob/main/nckh_crawldata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4
!pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json  # Nếu cần xử lý dữ liệu JSON
import re
from urllib.parse import urljoin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# mã cổ phiếu cần theo dõi tin tức
cp = 'VIC'

In [ ]:
url_cafef = f'https://cafef.vn/du-lieu/tin-doanh-nghiep/{cp}/event.chn'
url_vneconomy = f'https://vneconomy.vn/tim-kiem.htm?q={cp}'

Crawl data cho CafeF

In [ ]:
def crawl_page(url, session, save_path):
    response = session.get(url)
    print(f"Response status code: {response.status_code}")
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm thẻ div có id là divEvents
    div_events = soup.find('div', id='divEvents')
    print(f"div_events found: {div_events is not None}")

    # Tìm tất cả các thẻ <li> trong thẻ divEvents
    events = div_events.find_all('li')
    print(f"Number of events found: {len(events)}")

    for event in events:
        # Phần xử lý event giữ nguyên như cũ
        time_title = event.find('span', class_='timeTitle').get_text(strip=True)
        docnhanh_title_tag = event.find('a', class_='docnhanhTitle')
        docnhanh_title = docnhanh_title_tag['title']
        print(f"Event found: {time_title} - {docnhanh_title}")
        # đặt cấu trúc tên file
        file_name_not_final = re.sub(r"""[\\/*?:_,"<>'`‘’~|]""", ' ', f"{time_title}_{docnhanh_title}")
        file_name = file_name_not_final[:150].strip() + ".txt"

        href = docnhanh_title_tag['href']
        print(f"docnhanh_title_tag: {docnhanh_title_tag}")
        print(f"Article href: {href}")
        article_url = f"https://cafef.vn{href}"
        print(f"Article URL: {article_url}")

        article_response = session.get(article_url)
        print(f"Article response status code: {article_response.status_code}")

        if article_response.status_code != 200:
            print(f"Failed to retrieve article: {article_url}")
            continue

        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        paragraphs_div = article_soup.find('div', class_='detail-content afcbc-body')
        if paragraphs_div is None:
            print(f"No paragraphs found in article: {article_url}")
            continue

        paragraphs = paragraphs_div.find_all('p')
        if not paragraphs:
            print(f"No <p> tags found in article: {article_url}")
            continue

        print(f"Number of paragraphs found: {len(paragraphs)}")

        with open(os.path.join(save_path, file_name), 'w', encoding='utf-8') as file:
            for paragraph in paragraphs:
                file.write(paragraph.get_text(strip=True) + '\n')
                print(f"Paragraph written: {paragraph.get_text(strip=True)}")

    # Tạo URL cho trang tiếp theo sử dụng đúng AJAX endpoint
    page_index = url.split('PageIndex=')[1].split('&')[0] if 'PageIndex=' in url else '1'
    next_page = int(page_index) + 1
    next_url = f"https://cafef.vn/du-lieu/Ajax/Events_RelatedNews_New.aspx?symbol={cp}&floorID=0&configID=0&PageIndex={next_page}&PageSize=30&Type=2"

    return next_url

# Thiết lập và chạy crawler
save_path = '/content/drive/MyDrive/NCKH-2025/data_raw_cafef'
os.makedirs(save_path, exist_ok=True)

# URL ban đầu
initial_url = f"https://cafef.vn/du-lieu/Ajax/Events_RelatedNews_New.aspx?symbol={cp}&floorID=0&configID=0&PageIndex=1&PageSize=30&Type=2"

with requests.Session() as session:
    current_url = initial_url
    for page in range(1, 5):  # Crawl 10 trang, bạn có thể điều chỉnh số này
        print(f"\nCrawling page {page}")
        current_url = crawl_page(current_url, session, save_path)
        if not current_url:
            print("No more pages to crawl")
            break
        # Thêm delay giữa các request để tránh quá tải server
        time.sleep(2)

Crawl data cho vn economy

In [ ]:
import requests
import os
from bs4 import BeautifulSoup

# Setup
search_url = 'https://vneconomy.vn/'
save_path = '/content/drive/MyDrive/NCKH-2025/data_raw_vneconomy'
os.makedirs(save_path, exist_ok=True)

# Function to save article content to a file
def save_article(title, published_date, url_article, session):
    article_url = search_url + url_article
    print(f"Fetching article: {article_url}")  # Debug: Print URL being fetched
    response = session.get(article_url)
    print(f"Status code: {response.status_code}")  # Debug: Print status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        detail_content = soup.find('div', class_='detail__content')
        if detail_content:
            paragraphs = detail_content.find_all('p')
            if paragraphs:
                # Tạo tên file an toàn
                safe_title = "".join(c if c.isalnum() or c in [' ', '-', '_'] else '_' for c in title)
                filename = f"{published_date} {safe_title[:100]}.txt"
                filepath = os.path.join(save_path, filename)
                try:
                    with open(filepath, 'w', encoding='utf-8') as file:
                        for paragraph in paragraphs:
                            text = paragraph.get_text(strip=True)
                            if text:
                                file.write(text + '\n\n')
                                print(f"Paragraph written: {text}")  # Debug: Print each paragraph text
                    print(f"Saved: {filename}")
                except Exception as e:
                    print(f"Failed to save file {filename}: {e}")
            else:
                print(f"No paragraphs found in article: {article_url}")
        else:
            print(f"No detail content found in article: {article_url}")
    else:
        print(f"Failed to retrieve article: {article_url}")

# Main function to crawl data
def crawl_data(base_url, num_pages):
    session = requests.Session()
    for page in range(1, num_pages + 1):
        url = base_url + str(page)
        print(f"Fetching page: {url}")  # Debug: Print URL being fetched
        response = session.get(url)
        print(f"Status code: {response.status_code}")  # Debug: Print status code
        if response.status_code == 200:
            try:
                data = response.json().get('List', [])
                for item in data:
                    print(f"Item: {item}")  # Debug: Print each item data
                    title = item.get('Title')
                    published_date = item.get('PublishedDate')
                    url_article = item.get('UrlArticle')
                    if title and published_date and url_article:
                        save_article(title, published_date, url_article, session)
            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON from {url}: {e}")
        else:
            print(f"Failed to retrieve data from: {url}")

# Start crawling
num_pages = 10  # Số lần lặp lại quá trình crawl
base_url = f'https://search.hemera.com.vn/search/1/VIC/time/'  # Thay đổi mã CP theo nhu cầu
crawl_data(base_url, num_pages)